In [1]:
# Select NBA team to generate minute map
team_name = "WAS"

# 30 NBA teams to choose from:
# ATL  BRK  BOS  CHI  CHO
# CLE  DAL  DEN  DET  GSW
# HOU  IND  LAC  LAL  MEM
# MIA  MIL  MIN  NOP  NYK
# OKC  ORL  PHI  PHO  POR
# SAC  SAS  TOR  UTA  WAS

In [2]:
import pandas as pd
import folium
from math import sqrt

In [3]:
# record of all player seasons since 1998
players_past = pd.read_csv("nba_seasons.csv", sep=",", encoding = "latin") # use latin for non-English characters
players_past = players_past[["Player","ID", "Tm", "MP", "Year"]]

# remove rows not specific to one team (ie. traded player totals)
players_past = players_past[players_past["Tm"] != "TOT"]

# change defunct teams to current equivalent (ex. Seattle Supersonics -> OKC Thunder)
players_past["Tm"] = players_past["Tm"].replace(["CHA", "CHH"], "CHO")
players_past["Tm"] = players_past["Tm"].replace(["NOH", "NOK"], "NOP")
players_past["Tm"] = players_past["Tm"].replace("NJN", "BRK")
players_past["Tm"] = players_past["Tm"].replace("SEA", "OKC")
players_past["Tm"] = players_past["Tm"].replace("VAN", "MEM")

In [4]:
# record of all player seasons in 2019-20
players_current = pd.read_csv("current_players.csv", sep=",", encoding = "latin") # use latin for non-English characters

# remove rows not specific to one team (ie. traded player totals)
players_current = players_current[players_current["Tm"] != "TOT"]

In [5]:
# import location and color info for each team
arenas = pd.read_csv("arenas.csv")
cities = arenas[["Tm", "lat", "long", "Color"]]

In [6]:
# generate dataframe of all seasons from team's current players
team_id_list = list(players_current[players_current["Tm"] == team_name]["ID"])
team_players = players_past.loc[players_past["ID"].isin(team_id_list)]
team_players = team_players.sort_values(by=["Tm", "Year"])
team_players = team_players.astype({"MP": "int"})

In [7]:
# sum data and join location info to be used in map creation
map_data = team_players.groupby(["Tm"]).sum()
map_data = pd.merge(map_data, cities, on="Tm", how="left")
map_data = map_data.drop(columns=["Year"])

In [8]:
# create map centered on US
nba_map = folium.Map(location=[40.130, -98.35], zoom_start=4)

# create bubble and marker for each team in data
team_locations = folium.map.FeatureGroup()
for i in range(len(map_data)):
    team_locations.add_child(
        folium.vector_layers.CircleMarker(
            location = [map_data.iloc[i]['lat'], map_data.iloc[i]['long']], # coordinates of team's arena
            popup = str(map_data.iloc[i]['Tm'] + ": " + str((map_data.iloc[i]['MP'])) + " min"),
            radius = sqrt(map_data.iloc[i]['MP'])*0.2, # area of circle proportionate to number of minutes
            color = map_data.iloc[i]['Color'],
            fill = True,
            fill_color = map_data.iloc[i]['Color'],
            fill_opacity = 0.4
        )
    )
nba_map.add_child(team_locations)

# display map
nba_map

In [9]:
# save map for specific team
file_name = str("Maps/" + team_name + "_map.html")
nba_map.save(file_name)